In [1]:
import pandas as pd
from IPython.core.display import display, HTML
import matplotlib.pyplot as plt

%reload_ext autoreload
%autoreload 1

pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')


try:
    import scrapy
    from newspaper import Article
except:
    !pip install scrapy
    !pip install newspaper3k
    import scrapy
    from newspaper import Article

import pandas as pd
from urllib.request import urlopen, build_opener
import urllib
import pytz # time
from datetime import datetime
import requests
import re # regex
from tqdm import tqdm_notebook as tqdm
import pickle
from scrapy.crawler import CrawlerProcess
import time
import logging

     |████████████████████████████████| 245kB 2.9MB/s 
     |████████████████████████████████| 2.3MB 8.8MB/s 
     |████████████████████████████████| 3.1MB 22.4MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 235kB 39.1MB/s 
     |████████████████████████████████| 3.2MB 43.4MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11515 sha256=0041c9f82d1b60c3a68f9b7120bf04ccbaf81ce7d25923dfa20efb509c49d7f8
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7765 sha256=9b0e03e1b1d59ae67519dcb999afd1e9e9f26e0ec77f8e484a5b2d3e8e815b29
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
Successfully built PyDispatcher protego
     |████████████████████████████████| 215k

In [0]:
test_df = {
    'Source': ['FierceBiotech'],#, 'Barrons', 'InvestorPlace'],
    'Initial_URL': ['https://www.fiercebiotech.com/search-results/biotech?filter=published_date'],
               #    'https://www.barrons.com/search?keyword=',
               #    'www.investorplace.com/search/?q='],
    'URL_xpath': ['//*[@id="block-tektite-biotech-content"]/div[3]/div[1]/article/h2/a/@href'],#,
               #   '/html/body/section/div/div[2]/div/div[1]/div/div/div[2]/div/div[2]/div/div/ul/li/a/@href',
               #   '/html/body/div[2]/div[6]/div[1]/div/div[2]/div[5]/div/div/h3/a/@href'],
    'Older_posts_xpath': ['//*[@id="block-tektite-biotech-content"]/div[3]/div[2]/nav/ul/li[11]/a'],#,
                   #      '/html/body/section/div/div[2]/div/div[1]/div/div/div[2]/div/div[3]/a[11]/@href',
                   #      '/html/body/div[2]/div[6]/div[1]/div/div[2]/a[7]/@href'],
    'Publish_date_xpath': ['//*[@id="block-tektite-biotech-content"]/article/footer/div/div/time/text()']#,
                   #        '/html/body/div[3]/div[4]/div[2]/article/header/div[2]/div[1]/time',
                   #        '/html/body/main/section[2]/div/section/header/div/span[2]/abbr'] 
}
test_df = pd.DataFrame(test_df)


test_keyword = [

'biotech']

In [3]:
test_df

,Source,Initial_URL,URL_xpath,Older_posts_xpath,Publish_date_xpath
0,FierceBiotech,https://www.fiercebiotech.com/search-results/biotech?filter=published_date,"//*[@id=""block-tektite-biotech-content""]/div[3]/div[1]/article/h2/a/@href","//*[@id=""block-tektite-biotech-content""]/div[3]/div[2]/nav/ul/li[11]/a","//*[@id=""block-tektite-biotech-content""]/article/footer/div/div/time/text()"


In [4]:
URLs = []

class Scraper(scrapy.Spider):
  name = 'mySpider'
  custom_settings = {
        'DEPTH_LIMIT':  6,
        'LOG_LEVEL': logging.WARNING
        }

  def __init__(self, start_url, keyword, url_xpath, older_posts_xpath, pdate_xpath, source, *args, **kwargs):
    super(Scraper, self).__init__(*args, **kwargs)
    self.keyword = keyword
    self.start_urls = [start_url+ keyword]
    self.url_xpath = url_xpath
    self.older_posts_xpath = older_posts_xpath
    self.pdate_xpath = pdate_xpath
    self.source = source

  def parse(self, response):
    for article_url in response.xpath(self.url_xpath).extract():
      yield response.follow(article_url, callback=self.parse_article)
    older_posts = response.xpath(self.older_posts_xpath).extract_first()
    if older_posts is not None:
      yield response.follow(older_posts, callback=self.parse)

  def parse_article(self, response):
    try:
      publish_date = response.xpath(self.pdate_xpath).extract()
    except:
      publish_date = float('nan')

    global URLs
    URLs.append({
        'Url': response.request.url,
        'Publish_Date': publish_date,
        "Keyword": self.keyword,
        'Source': self.source
        })
  
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 Safari/537.36'
})

for i in range(len(test_df)):
  for keyword in test_keyword:
    process.crawl(Scraper,
                start_url= test_df.loc[i, "Initial_URL"],
                keyword = keyword,
                url_xpath = test_df.loc[i, "URL_xpath"],
                older_posts_xpath = test_df.loc[i, "Older_posts_xpath"],
                pdate_xpath = test_df.loc[i, "Publish_date_xpath"],
                source = test_df.loc[i, "Source"]
                )  

process.start()

scrapy_url_df = pd.DataFrame(URLs)

2020-04-01 21:20:23 [scrapy.utils.log] INFO: Scrapy 2.0.1 started (bot: scrapybot)
2020-04-01 21:20:23 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.6.9 (default, Nov  7 2019, 10:44:02) - [GCC 8.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.14.137+-x86_64-with-Ubuntu-18.04-bionic
2020-04-01 21:20:23 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-04-01 21:20:23 [scrapy.crawler] INFO: Overridden settings:
{'DEPTH_LIMIT': 6,
 'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) '
               'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.87 '
               'Safari/537.36'}


In [5]:
scrapy_url_df

,Url,Publish_Date,Keyword,Source
0,https://www.fiercebiotech.com/special-report/axial-biotherapeutics,"[Sep 26, 2017 7:00am]",biotech,FierceBiotech
1,https://www.fiercebiotech.com/biotech/top-women-biopharma-for-2016-a-call-for-nominations,"[Jul 26, 2016 2:54pm]",biotech,FierceBiotech
2,https://www.fiercebiotech.com/cro/alzprotect-pens-parexel-brain-disease-pact-using-new-biotech-division,"[Mar 25, 2019 9:50am]",biotech,FierceBiotech
3,https://www.fiercebiotech.com/biotech/novartis-farms-out-rare-disease-drug-to-lifemax-labs,"[Aug 27, 2018 10:30am]",biotech,FierceBiotech
4,https://www.fiercebiotech.com/biotech/phase-2-update-krystal-biotech-eyes-phase-3-for-rare-skin-disease-gene-therapy,"[Jun 24, 2019 11:13am]",biotech,FierceBiotech
5,https://www.fiercebiotech.com/biotech/takeda-joins-car-t-chasing-pack-noile-immune-alliance,"[Sep 5, 2017 11:23am]",biotech,FierceBiotech
6,https://www.fiercebiotech.com/biotech/lilly-cmo-robert-brown-to-lead-brickell-bio-as-it-eyes-phase-3-dermatology-trials,"[Nov 19, 2018 3:32pm]",biotech,FierceBiotech
7,https://www.fiercebiotech.com/medical-devices/alzheimer-s-biochip-blood-test-as-accurate-as-molecular-dx,"[Aug 3, 2016 3:45pm]",biotech,FierceBiotech
8,https://www.fiercebiotech.com/fiercebiotech-cro-newsletter-sample,[],biotech,FierceBiotech
9,https://www.fiercebiotech.com/page-not-found,[],biotech,FierceBiotech


In [6]:
articles = []
url_regex_pattern = re.compile('https?://(www\.)?([\w\.\-\_]+\.\w+)')

url_df = scrapy_url_df

for i in tqdm(range(len(url_df))): 

  url = url_df['Url'][i]

  source_match = url_regex_pattern.match(url)
  source = source_match.group(2)

  keyword = url_df['Keyword'][i]

  try:
    article = Article(url)
    article.download()
    article.parse()

    try:
      publish_date = url_df['Publish_Date'][i]
    except KeyError:
      publish_date = article.publish_date

    articles.append({
      'Title': article.title,
      'Text': article.text,
      'Publish_Date': publish_date,
      'URL': url,
      'Scrape_Date': datetime.now(pytz.timezone('US/Central')).strftime("%x",),
      'Source': source,
      'Search_Keyword': keyword
      })
    
  except newspaper.ArticleException: # webpage not found
    articles.append({
      'Title': float('nan'),
      'Text': float('nan'),
      'Publish_Date': float('nan'),
      'URL': url,
      'Scrape_Date': datetime.now(pytz.timezone('US/Central')).strftime("%x",),
      'Source': source,
      'Search_Keyword': keyword
      })

In [7]:
articles_df = pd.DataFrame(articles)
print(articles_df.shape)
articles_df.head()
articles_df.to_csv('last.csv')

(20, 7)


In [8]:
articles_df.iloc[0,1]

"This biopharmaceutical company is targeting the gut to heal the brain.\n\nCEO: David Donabedian\n\nFounded: 2016\n\nBased: Boston\n\nClinical focus: Diseases and disorders of the central nervous system\n\nThe scoop: Axial is working to treat CNS disorders in a new way—one that doesn’t involve the blood-brain barrier. Instead, the biotech has a three-pronged approach to treat disease through the gut-brain connection. This includes small-molecule and biologic treatments, as well as bacteria-based therapies to modify the gut microbiome and affect changes in the brain.\n\nWhat makes Axial Fierce: A number of diseases and disorders fall under the CNS umbrella, ranging from mood disorders and neurodegenerative diseases such as Alzheimer’s and Parkinson’s to conditions caused by infection or trauma.\n\nThe blood-brain barrier protects the brain from infection but also stops nearly all small and large molecules from entering the organ, which has hindered efforts to develop neurotherapeutic dr

In [9]:
filepath = '/content/gdrive/My Drive/'
articles_df.to_csv(filepath+'scrapy_10_26.csv', index=False)

pickle.dump(articles_df,open(filepath+'articles_df.p','wb'))

FileNotFoundError: ignored